In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import  cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog, local_binary_pattern
from sklearn.preprocessing import StandardScaler
from natsort import natsorted


In [ ]:
genuine_path = r'archive(1)\signatures\full_org'
forged_path = r'archive(1)\signatures\full_forg'


In [89]:

def extract_features(img):
    img = cv2.resize(img, (256, 128))
    _, binary_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if np.sum(binary_img) < (binary_img.size * 255 / 2):
        binary_img = cv2.bitwise_not(binary_img)
    hog_features = hog(img, 
                      pixels_per_cell=(16, 16),
                      cells_per_block=(2, 2),
                      feature_vector=True)
    
    # 2. LBP Features (from grayscale image)
    lbp = local_binary_pattern(binary_img, P=24, R=3, method="uniform")
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    lbp_hist = lbp_hist.astype("float") / (lbp_hist.sum() + 1e-7)  # Add epsilon to avoid division by zero
    
    # 3. Hu Moments (from binary image)
    moments = cv2.moments(binary_img)
    hu_moments = cv2.HuMoments(moments).flatten()
    
    # Log-scale normalization for Hu Moments
    hu_moments = np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-7)
    
    # Combine all features
    features = np.hstack([hog_features, lbp_hist, hu_moments])
    
    return features


In [ ]:

def load_images(folder, label):
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    file_list = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    data_size = len(file_list)
    train_size = int(0.8 * data_size)
    file_list = natsorted(file_list)  # Sort files naturally
    
    for i, filename in enumerate(file_list):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            features = extract_features(img)
            if i < train_size:
                X_train.append(features)
                y_train.append(label)
            else:
                X_test.append(features)
                y_test.append(label)
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)


In [92]:
real_X_train,real_X_test,real_y_train,real_y_test=load_images(genuine_path,label=0)
forged_X_train,forged_X_test,forged_y_train,forged_y_test=load_images(forged_path,label=1)


In [93]:
X_train=np.concatenate((real_X_train,forged_X_train), axis=0)
X_test=np.concatenate((real_X_test,forged_X_test), axis=0)
y_train=np.concatenate((real_y_train,forged_y_train), axis=0)
y_test=np.concatenate((real_y_test,forged_y_test), axis=0)


In [94]:
train_data = np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)
test_data = np.concatenate((X_test, y_test.reshape(-1, 1)), axis=1)


In [95]:
np.random.shuffle(train_data)  # Shuffles in-place
np.random.shuffle(test_data)

In [96]:

X_train_shuffled = train_data[:, :-1]
y_train_shuffled = train_data[:, -1]

X_test_shuffled = test_data[:, :-1]
y_test_shuffled = test_data[:, -1]


In [97]:
scaler = StandardScaler()
X_train_shuffled = scaler.fit_transform(X_train_shuffled)
X_test_shuffled = scaler.transform(X_test_shuffled)


In [98]:

svm = SVC(kernel='linear', C=100.0)
svm.fit(X_train_shuffled, y_train_shuffled)
y_pred_svm = svm.predict(X_test_shuffled)


In [99]:
print("SVM Accuracy:", accuracy_score(y_test_shuffled, y_pred_svm))

SVM Accuracy: 0.9754716981132076


In [100]:
rf = RandomForestClassifier(n_estimators=100, random_state=55)
rf.fit(X_train_shuffled, y_train_shuffled)
y_pred_rf = rf.predict(X_test_shuffled)

In [101]:
print("Random Forest Accuracy:", accuracy_score(y_test_shuffled, y_pred_rf))

Random Forest Accuracy: 0.969811320754717
